In [ ]:
pip install pandas numpy matplotlib seaborn scikit-learn mysql-connector-python

In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
import mysql.connector
import pyodbc


In [ ]:
!pip install sqlalchemy pymysql
import joblib
from sqlalchemy import create_engine

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('machine failure.csv')

# Display the first few rows
print(df.head())

print(df.describe())
print(df.info())

In [ ]:
print(df.isnull().sum())
df = df.dropna()  # or use data.fillna() to fill missing values

In [114]:
df = pd.get_dummies(df)

In [115]:
X = df.drop('Machine failure', axis=1)  # Replace 'target_column' with your actual target column
y = df['Machine failure']

In [116]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier()
gb_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

rf_predictions = rf_model.predict(X_test)
print(confusion_matrix(y_test, rf_predictions))
print(classification_report(y_test, rf_predictions))

In [ ]:
gb_predictions = gb_model.predict(X_test)
print(confusion_matrix(y_test, gb_predictions))
print(classification_report(y_test, gb_predictions))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cm_rf = confusion_matrix(y_test, rf_predictions)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues')
plt.title('Random Forest Confusion Matrix')
plt.show()

cm_gb = confusion_matrix(y_test, gb_predictions)
sns.heatmap(cm_gb, annot=True, fmt='d', cmap='Reds')
plt.title('Gradient Boosting Confusion Matrix')
plt.show()

In [122]:
import joblib

joblib.dump(rf_model, 'random_forest_model.pkl')
joblib.dump(gb_model, 'gradient_boosting_model.pkl')

['gradient_boosting_model.pkl']

In [123]:
rf_model = joblib.load('random_forest_model.pkl')
gb_model = joblib.load('gradient_boosting_model.pkl')

In [ ]:
# Make predictions on the entire dataset
rf_predictions = rf_model.predict(X)
gb_predictions = gb_model.predict(X)

# Add predictions to the dataframe
df['rf_predictions'] = rf_predictions
df['gb_predictions'] = gb_predictions

# Select necessary columns for insertion
predictions_df = df[['UDI', 'rf_predictions', 'gb_predictions']]

# Rename columns if necessary to match your database schema
predictions_df = predictions_df.rename(columns={'rf_predictions': 'rf_prediction', 'gb_predictions': 'gb_prediction'})

predictions_df

In [129]:
from sqlalchemy import create_engine, Table, Column, Float, Integer, MetaData, DateTime, insert, func
from sqlalchemy.orm import sessionmaker

# Create the SQLAlchemy engine
engine = create_engine(connection_string)
metadata = MetaData()

# Define the table schema
predictions_table = Table(
    'predictions_table', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('TWF', Float),
    Column('HDF', Float),
    Column('PWF', Float),
    Column('OSF', Float),
    Column('RNF', Float),
    Column('rf_prediction', Integer),
    Column('gb_prediction', Integer),
    Column('timestamp', DateTime, default=func.now())
)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Prepare data for insertion
predictions_df = df[['TWF', 'HDF', 'PWF', 'OSF', 'RNF', 'rf_predictions', 'gb_predictions']]
predictions_df = predictions_df.rename(columns={'rf_predictions': 'rf_prediction', 'gb_predictions': 'gb_prediction'})

# Convert DataFrame to dictionary format for insertion
data = predictions_df.to_dict(orient='records')

try:
    # Insert data into the table
    with engine.connect() as conn:
        conn.execute(insert(predictions_table), data)
    # Commit the transaction
    session.commit()
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    session.close()

In [ ]:
from sqlalchemy import select

# Create a new session
with engine.connect() as conn:
    result = conn.execute(select(predictions_table))
    for row in result:
        print(row)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Load your dataset
data = pd.read_csv('machine failure.csv')

# Prepare features and target variable
X = data[['TWF', 'HDF', 'PWF', 'OSF', 'RNF']]
y = data['Machine failure']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# Save the model
joblib.dump(model, 'machine_failure_model.pkl')

In [ ]:
import joblib
import pandas as pd

# Load the trained model
model = joblib.load('machine_failure_model.pkl')

# Example input data
new_data = pd.DataFrame({
    'TWF': [0],
    'HDF': [1],
    'PWF': [0],
    'OSF': [0],
    'RNF': [1]
})

# Predict machine failure
predictions = model.predict(new_data)
print(predictions)  # This will print 0 or 1